In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import ElasticNet,.ElasticNetCV
from sklearn import model_selection as ms
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn import metrics

from warnings import simplefilter

rand_st=4
simplefilter("ignore") 

In [ ]:
elos=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\clean_elos.csv')
elos.head()

In [ ]:
features=pd.read_csv(r'C:\Users\Asus\PredictingRatings\data\features.csv')
print(features.info())
print(features.shape)
features.describe()

In [ ]:
target_white=elos['WhiteElo']
target_black=elos['BlackElo']
target_mean=elos['MeanElos']
target_diff=elos['DiffElos']
target_sum=elos['SumElos']

In [ ]:
features1=features[:500]
target_sum1=target_sum[:500]

In [ ]:
X_train, X_test, Y_train, Y_test =ms.train_test_split(features1, target_sum1, test_size = 0.3,random_state=rand_st)
print ('Train data size: {} instances \nTest data size: {} instances'.format(len(X_train), len(Y_train)))
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
def predictions_report(pred_target,predictions):
    mae_report=mae(target_sum,sum_pred)
    print ('MAE = {:.3f}'.format(mae_report))
    
    rmse_report = (mse(target_sum, sum_pred))**0.5
    print ('RMSE = {:.3f}'.format(rmse_report))
    
    corr_coef = pearsonr(target_sum1,sum_pred)
    print("Correlation coefficient = {:.3f}".format(corr_coef[0]))
    
    return (mae_report,rmse_report,corr_coef)

In [ ]:
def GridTune(params,classifiers,folds):
    mae = make_scorer(metrics.mean_absolute_error)
   
    grid = ms.GridSearchCV(classifiers,
                           params,  # настройка параметров через кросс-валидацию
                           refit=True,    # переобучение по лучшим найденным параметрам   
                           scoring=mae,
                           n_jobs=-1,
                           cv=ms.StratifiedKFold(n_splits=folds))

    
    grid_best_model = grid.fit(X_train,Y_train)

    print("Лучшие параметры для MAE:")
    print(grid_best_model.best_params_)
    print('\nЛучший алгоритм:')
    print(grid_best_model.best_estimator_)
    return grid_best_model

In [ ]:
lr_params =[]

start = time.time()
lr=LinearRegression(random_state=rand_st,n_jobs=-1')
lr_model=GridTune(lr_params,lr,2)
lr_predictions=lr_model.predict(X_test)
scores=predictions_report(Y_test,lr_predictions)
full_time=round((time.time() - start)/2,3)
print(full_time, 'c')

In [ ]:
print ('MAE = {:.3f}'.format(scores[0]))
print ('RMSE = {:.3f}'.format(scores[1]))
print("Correlation coefficient = {:.3f}".format(scores[2]))

In [ ]:
output=pd.read_table(r'C:\Users\Asus\PredictingRatings\models\results.xlsx')

In [ ]:
output = output.append(pd.Series([str(lr_1), mae1,rmse1, 
                                  corr_coef1[0],full_time1,'Обучение только на победах белых -> прогноз ретйтинга белых'],
                                 index=output.columns ),
                      ignore_index=True)

output

In [ ]:
output.to_excel(r'C:\Users\Asus\PredictingRatings\models\results.xlsx',index=False)